In [1]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

load_dotenv(PARENT_DIR / ".env")

HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
HOPSWORKS_PROJECT_NAME = os.getenv("HOPSWORKS_PROJECT_NAME")

In [2]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2024
to_year = 2025
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    
    # download data for the whole year
    rides_one_year = load_raw_data(year)
    
    # append rows
    rides = pd.concat([rides, rides_one_year])

File 2024-01 was already in local storage
File 2024-02 was already in local storage
File 2024-03 was already in local storage
File 2024-04 was already in local storage
File 2024-05 was already in local storage
File 2024-06 was already in local storage
File 2024-07 was already in local storage
File 2024-08 was already in local storage
File 2024-09 was already in local storage
File 2024-10 was already in local storage
File 2024-11 was already in local storage
File 2024-12 was already in local storage
File 2025-01 was already in local storage
File 2025-02 was already in local storage
File 2025-03 was already in local storage
2025-04 file is not available
2025-05 file is not available
2025-06 file is not available
2025-07 file is not available
2025-08 file is not available
2025-09 file is not available
2025-10 file is not available
2025-11 file is not available
2025-12 file is not available


In [3]:
print(f'{len(rides)=:,}')

len(rides)=52,367,240


In [4]:
from src.data import transform_raw_data_into_ts_data


ts_data = transform_raw_data_into_ts_data(rides)
ts_data = ts_data.rename(columns={"rides": "rides_count"})
ts_data

100%|██████████| 265/265 [00:05<00:00, 48.53it/s]


,pickup_hour,rides_count,pickup_location_id,rides_count
0,2024-01-01 00:00:00,0.0,1,NaN
1,2024-01-01 01:00:00,0.0,1,NaN
2,2024-01-01 02:00:00,0.0,1,NaN
3,2024-01-01 03:00:00,0.0,1,NaN
4,2024-01-01 04:00:00,0.0,1,NaN
...,...,...,...,...
2900155,2025-03-31 19:00:00,2.0,265,NaN
2900156,2025-03-31 20:00:00,0.0,265,NaN
2900157,2025-03-31 21:00:00,2.0,265,NaN
2900158,2025-03-31 22:00:00,1.0,265,NaN


In [5]:
import hopsworks

In [ ]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

In [7]:
feature_store = project.get_feature_store()

In [8]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [9]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [ ]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})